In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [18]:
%run -i ../../core.py

In [19]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [20]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.sqrt(torch.var(dummy_data_tensor, correction = 0))
dummy_data_mean = torch.mean(dummy_data_tensor)

In [21]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        # since we square
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*X.size()[1])<=torch.abs(0.02*self.w*self.w*X.size()[1]),x_mean_cons ),self.w)

        # return (torch.logical_and(torch.abs(torch.sqrt(torch.sum((X-self.data_mean)*(X-self.data_mean)))-self.w*torch.sqrt(X.size()[1]))<0.01*self.w*torch.sqrt(X.size()[1]),x_mean_cons ),self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [22]:
# Prover/ data owner side
theory_output = torch.sqrt(torch.var(data_tensor, correction = 0))
data_mean = torch.mean(data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        # since we square
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*X.size()[1])<=torch.abs(0.02*self.w*self.w*X.size()[1]),x_mean_cons ),self.w)
        # return (torch.logical_and(torch.abs(torch.sqrt(torch.sum((X-self.data_mean)*(X-self.data_mean)))-self.w*torch.sqrt(X.size()[1]))<0.01*self.w*torch.sqrt(X.size()[1]),x_mean_cons ),self.w)
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

theory output:  tensor(14.5557)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-40670,8],"logrows":16,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1520,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [23]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 5.533793687820435 seconds
Theory output:  tensor(14.5557)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 14.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]], 'proof': '153f0119aae34d625aa02c123b96398b3d6caea2501a1f54ae5f7d03be9d0e8b1783fd4f7a623af19df76b3cec31c2b194b9d7523449d9290f3c71a3892742f90bfc6924ccb76025610fdfb978458a903f71a1a7c77240fc7a857dde910032cd03b4acc3f75955b06292cffdfd8293cf82e1082c4e021c2e39d53cd84844574e0f5b938b36cfbddca0bf0b560dc01eb51e0033b2e86271e3a26f423ddd9d750109158fb3faa5de76a401cdc68510e326e84afc46e6405ead1735d3187822757f0bc6d834b2971715220ad723ff972d7024ea0dae3244af7421520e3c0e7a16a120985d00ebba3cf28cad27a808acaabf57aa4668aadcd609e830bde6545e6a571c9a67c20b68f49d1cfa789bd5b5865deb6546241a0fc3e2a2e54eb3635a05c524ea36016978ae0c258bf3ebeb0f322172fdfb2f12e9c62bd6f7db4905b458ee24c2e82b9d3dee4d6e43ddc7d0fceeaea87535b1aca6f1f9941cff24ead2

In [24]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]]
proof boolean:  1.0
proof result 1 : 14.5
verified
